In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [143]:
class Student:
    
    def __init__(self, name, gender):
        self.name = name
        self.gender = gender
        
    def set_age_and_year(self, age = 0):
        
        """
        This function is used to set the age and the year of study of the university student
        
        Args:
            age (int, default = 0): The age of the student
        
        """
        self.age = age
        list_age = [18,19,20,21] # The age group of university student (usually from age 18 - 21)
        if self.age in list_age: # Test if the input age is inside the list, list_age
            self.year = int(list_age.index(self.age)) + 1
        else:
            self.year = 0 # Non university student
            print('The age below 18 are not university student')
        
    def set_major(self, major):
        
        """
        This function is to set the major of the student
        
        Args:
            major (str): The major of the university student
        
        """
        if self.year == 0 and major == 'no': # For non university student
            print('The age below 18 are not university student')
            self.major = 'null'
            
        elif self.year == 1 and major == 'no': # For year one student
            print('Year one student has not declare major')
            self.major = 'null'
            
        else:
            self.major = major # set the input major to self.major
        
    def gpa_calculate(self, credit, grade, university):
        
        """
        To set the gpa of the university student
        
        Args:
            credit (list): a list containing the credit for all the subjects. 
                           e.g. A student has four '3-creds',then courses = [3,3,3,3]
                           
            grade (list): a list containing the grade for all the subjects
                          e.g. A student's grade point are 'A+, B, B+, C', then grade = ['A+', 'B', 'B+', 'C']
            
            university (str): The university of the student.
        """
        
        self.credit = credit
        self.grade = grade
        
        def get_url(url, university):
            
            """
            To get the url and the grade point of different universities.
            
            Args:
                url (str): The grading system url of different university in Hong Kong.
                
                university (str): The university of the student.
            
            Returns:
                numpy array: The grade and grade point for different universities. 
                             e.g. hkust has grade = array(['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D', 'F']),
                                  and grade point = array([4.3, 4, 3.7, 3.3, 3, 2.7, 2.3, 2, 1.7, 1, 0])
                
            """
            
            r = requests.get(url)
            html_doc = r.text
            
            # These universities have similar grading system (A+ = 4.3, A = 4, A- = 3.7)
            if university in ['hku', 'polyu', 'hkust', 'cityu', 'ouhk']:  
                
                df = pd.read_html(html_doc)[0]
                grade = np.array(df['Grade']['Grade'])
                grade_point = np.array(df['From 2020/21']['Corresponding Grade Point'])
                
                if university == 'hkust' or university == 'cityu':  # HKUST and CityU don't have D+
                    grade = np.delete(grade, 9)
                    grade_point = np.delete(grade_point, 9)
                    
                if university == 'ouhk': # OUHK don't have A+, C-, D+ and D
                    grade = np.delete(grade, [0,8,9,10])
                    grade_point = np.delete(grade_point, [0,8,9,10])
                    
                return grade, grade_point
            
            # These universities have similar grading system (A = 4.00, A- = 3.67, B+ = 3.33, *without D+)
            elif university == 'hkbu' or university == 'eduhk':
                df = pd.read_html(html_doc)[0]
                df = df.iloc[0:11]
                grade = np.array(df['Letter Grade'])
                grade_point = np.array(df['Grade Point per Unit'])
                
                if university == 'eduhk':
                    grade = np.append(grade, ['A+']) # EduHK has A+ as 4.33
                    grade_point = np.append(grade_point, ['4.33'])
                    
                return grade, grade_point
            
            # These universities have similar grading system (A = 4.0, A- = 3.7, B+ = 3.3, *with D+)
            elif university == 'cuhk' or university == 'hsuhk':
                df = pd.read_html(html_doc)[1]
                df = df.iloc[7:18,2:4]
                grade = np.array(df.iloc[:,0])
                grade_point = np.array(df.iloc[:,1])
                
                return grade, grade_point
            
            # These universities have similar grading system (A = 4.00, A- = 3.67, B+ = 3.33, *with D+)
            elif university == 'lingu':
                df = pd.read_html(html_doc)[0]
                grade = np.array(df.loc[0:10, 'Sub-divisions'])
                grade_point = np.array(df.loc[0:10, 'Grade Points'])
                
                return grade, grade_point
        
        # use the function "get_url" to get the grade and grade point for different universities.
        if university in ['hku', 'polyu', 'hkust', 'cityu', 'ouhk']:
            grades, grade_point = get_url('https://www.polyu.edu.hk/ar/ARN/arn_explanation.html', university)
            
        elif university == 'hkbu' or university == 'eduhk':
            grades, grade_point = get_url('https://ar.hkbu.edu.hk/student-services/academic-results/assessment-grading-system', university)
        
        elif university == 'cuhk' or university == 'hsuhk':
             grades, grade_point = get_url('http://www.res.cuhk.edu.hk/en-gb/general-information/guide-to-grading-system', university)
        
        elif university == 'lingu':
             grades, grade_point = get_url('https://www.ln.edu.hk/arts/first-year-students/grading-system.php', university)
        
        gpa = []
        for num in range(len(self.grade)):
            
            #Check if the input grade is inside the grading system of the university, if not return gpa = 0.00
            if self.grade[num] not in grades:
                print(f'Error!! {self.grade[num]} is not included in the grading system')
                gpa = [0.00]
                break
            
            """
            Since the position of grade is corresponding to particular grade point, we should use 'i' in 
            for loop to record the position
            
            If the input grade is equal to the grading system of the university, we can find the 
            particular grade point for the grade using 'i'. 
            """
            for i in range(len(grades)):
                if grades[i] == self.grade[num]:
                    point = float(grade_point[i])
                    gpa_ = round(point*self.credit[num], 3) # multiply the gpa wwith respect to the credit/weight
                    gpa.append(gpa_)
                    
        self.gpa = gpa
        cga = round(sum(gpa)/sum(self.credit), 3) # calculate the cga or semester gpa according to the credit/weight and grade
        self.cga = cga

# Testing

## HKU

In [99]:
credit_list = []
grade_list = []

n = int(input('Please input the number of course: '))
print('Please enter the credit and grade for each course: (Credit first, followed by the grade)')
for num in range(n):
    credit_input = int(input('Credit: '))
    grade_input = input('Grade: ')
    credit_list.append(credit_input)
    grade_list.append(grade_input)

Please input the number of course: 3
Please enter the credit and grade for each course: (Credit first, followed by the grade)
Credit: 3
Grade: A+
Credit: 3
Grade: B
Credit: 3
Grade: D+


In [101]:
print('HKU:')
stu1 = Student('Tang Ho Chun', 'M')
stu1.set_age_and_year(18)
stu1.set_major('no')
stu1.gpa_calculate(credit_list, grade_list, 'hku')
print(f'name: {stu1.name}, gender: {stu1.gender}')
print(f'major: {stu1.major}, year: {stu1.year}')
print(f'sem gpa: {stu1.cga}')

HKU:
Year one student has not declare major
name: Tang Ho Chun, gender: M
major: null, year: 1
sem gpa: 2.867


## HKUST 

In [111]:
credit_list = []
grade_list = []

n = int(input('Please input the number of course: '))
print('Please enter the credit and grade for each course: (Credit first, followed by the grade)')
for num in range(n):
    credit_input = int(input('Credit: '))
    grade_input = input('Grade: ')
    credit_list.append(credit_input)
    grade_list.append(grade_input)

Please input the number of course: 5
Please enter the credit and grade for each course: (Credit first, followed by the grade)
Credit: 3
Grade: B+
Credit: 3
Grade: B
Credit: 3
Grade: B
Credit: 3
Grade: A-
Credit: 3
Grade: C+


In [112]:
print('HKUST:')
stu1 = Student('Tang Ho Chun', 'M')
stu1.set_age_and_year(21)
stu1.set_major('IEDA')
stu1.gpa_calculate(credit_list, grade_list, 'hkust')
print(f'name: {stu1.name}, gender: {stu1.gender}')
print(f'major: {stu1.major}, year: {stu1.year}')
print(f'sem gpa: {stu1.cga}')

HKUST:
name: Tang Ho Chun, gender: M
major: IEDA, year: 4
sem gpa: 3.06


In [137]:
credit_list = []
grade_list = []

n = int(input('Please input the number of course: '))
print('Please enter the credit and grade for each course: (Credit first, followed by the grade)')
for num in range(n):
    credit_input = int(input('Credit: '))
    grade_input = input('Grade: ')
    credit_list.append(credit_input)
    grade_list.append(grade_input)

Please input the number of course: 2
Please enter the credit and grade for each course: (Credit first, followed by the grade)
Credit: 3
Grade: A
Credit: 3
Grade: D+


In [144]:
print('HKUST:')
stu1 = Student('Tang Ho Chun', 'M')
stu1.set_age_and_year(21)
stu1.set_major('IEDA')
stu1.gpa_calculate(credit_list, grade_list, 'hkust')
print(f'name: {stu1.name}, gender: {stu1.gender}')
print(f'major: {stu1.major}, year: {stu1.year}')
print(f'sem gpa: {stu1.cga}')

HKUST:
Error!! D+ is not included in the grading system
name: Tang Ho Chun, gender: M
major: IEDA, year: 4
sem gpa: 0.0


## CUHK 

In [150]:
credit_list = []
grade_list = []

n = int(input('Please input the number of course: '))
print('Please enter the credit and grade for each course: (Credit first, followed by the grade)')
for num in range(n):
    credit_input = int(input('Credit: '))
    grade_input = input('Grade: ')
    credit_list.append(credit_input)
    grade_list.append(grade_input)

Please input the number of course: 3
Please enter the credit and grade for each course: (Credit first, followed by the grade)
Credit: 3
Grade: A
Credit: 3
Grade: B+
Credit: 3
Grade: B


In [151]:
print('CUHK:')
stu1 = Student('Tang Ho Chun', 'M')
stu1.set_age_and_year(18)
stu1.set_major('no')
stu1.gpa_calculate(credit_list, grade_list, 'cuhk')
print(f'name: {stu1.name}, gender: {stu1.gender}')
print(f'major: {stu1.major}, year: {stu1.year}')
print(f'sem gpa: {stu1.cga}')

CUHK:
Year one student has not declare major
name: Tang Ho Chun, gender: M
major: null, year: 1
sem gpa: 3.433


In [145]:
credit_list = []
grade_list = []

n = int(input('Please input the number of course: '))
print('Please enter the credit and grade for each course: (Credit first, followed by the grade)')
for num in range(n):
    credit_input = int(input('Credit: '))
    grade_input = input('Grade: ')
    credit_list.append(credit_input)
    grade_list.append(grade_input)

Please input the number of course: 3
Please enter the credit and grade for each course: (Credit first, followed by the grade)
Credit: 3
Grade: A
Credit: 3
Grade: B+
Credit: 3
Grade: A+


In [148]:
print('CUHK:')
stu1 = Student('Tang Ho Chun', 'M')
stu1.set_age_and_year(20)
stu1.set_major('IEDA')
stu1.gpa_calculate(credit_list, grade_list, 'cuhk')
print(f'name: {stu1.name}, gender: {stu1.gender}')
print(f'major: {stu1.major}, year: {stu1.year}')
print(f'sem gpa: {stu1.cga}')

CUHK:
Error!! A+ is not included in the grading system
name: Tang Ho Chun, gender: M
major: IEDA, year: 3
sem gpa: 0.0


## PolyU 

In [153]:
credit_list = []
grade_list = []

n = int(input('Please input the number of course: '))
print('Please enter the credit and grade for each course: (Credit first, followed by the grade)')
for num in range(n):
    credit_input = int(input('Credit: '))
    grade_input = input('Grade: ')
    credit_list.append(credit_input)
    grade_list.append(grade_input)

Please input the number of course: 3
Please enter the credit and grade for each course: (Credit first, followed by the grade)
Credit: 3
Grade: A+
Credit: 3
Grade: B
Credit: 3
Grade: D+


In [154]:
print('PolyU:')
stu1 = Student('Tang Ho Chun', 'M')
stu1.set_age_and_year(20)
stu1.set_major('IEDA')
stu1.gpa_calculate(credit_list, grade_list, 'polyu')
print(f'name: {stu1.name}, gender: {stu1.gender}')
print(f'major: {stu1.major}, year: {stu1.year}')
print(f'sem gpa: {stu1.cga}')

PolyU:
name: Tang Ho Chun, gender: M
major: IEDA, year: 3
sem gpa: 2.867


## CityU

In [155]:
print('CityU:')
stu1 = Student('Tang Ho Chun', 'M')
stu1.set_age_and_year(20)
stu1.set_major('IEDA')
stu1.gpa_calculate(credit_list, grade_list, 'cityu')
print(f'name: {stu1.name}, gender: {stu1.gender}')
print(f'major: {stu1.major}, year: {stu1.year}')
print(f'sem gpa: {stu1.cga}')

CityU:
Error!! D+ is not included in the grading system
name: Tang Ho Chun, gender: M
major: IEDA, year: 3
sem gpa: 0.0


In [162]:
credit_list = []
grade_list = []

n = int(input('Please input the number of course: '))
print('Please enter the credit and grade for each course: (Credit first, followed by the grade)')
for num in range(n):
    credit_input = int(input('Credit: '))
    grade_input = input('Grade: ')
    credit_list.append(credit_input)
    grade_list.append(grade_input)

Please input the number of course: 3
Please enter the credit and grade for each course: (Credit first, followed by the grade)
Credit: 3
Grade: A+
Credit: 3
Grade: A
Credit: 3
Grade: B


In [163]:
print('CityU:')
stu1 = Student('Tang Ho Chun', 'M')
stu1.set_age_and_year(20)
stu1.set_major('IEDA')
stu1.gpa_calculate(credit_list, grade_list, 'cityu')
print(f'name: {stu1.name}, gender: {stu1.gender}')
print(f'major: {stu1.major}, year: {stu1.year}')
print(f'sem gpa: {stu1.cga}')

CityU:
name: Tang Ho Chun, gender: M
major: IEDA, year: 3
sem gpa: 3.767


## HKBU

In [164]:
print('HKBU:')
stu1 = Student('Tang Ho Chun', 'M')
stu1.set_age_and_year(20)
stu1.set_major('IEDA')
stu1.gpa_calculate(credit_list, grade_list, 'hkbu')
print(f'name: {stu1.name}, gender: {stu1.gender}')
print(f'major: {stu1.major}, year: {stu1.year}')
print(f'sem gpa: {stu1.cga}')

HKBU:
Error!! A+ is not included in the grading system
name: Tang Ho Chun, gender: M
major: IEDA, year: 3
sem gpa: 0.0


## EduHK

In [165]:
print('Eduhk:')
stu1 = Student('Tang Ho Chun', 'M')
stu1.set_age_and_year(20)
stu1.set_major('IEDA')
stu1.gpa_calculate(credit_list, grade_list, 'eduhk')
print(f'name: {stu1.name}, gender: {stu1.gender}')
print(f'major: {stu1.major}, year: {stu1.year}')
print(f'sem gpa: {stu1.cga}')

Eduhk:
name: Tang Ho Chun, gender: M
major: IEDA, year: 3
sem gpa: 3.777


## OUHK

In [166]:
credit_list = []
grade_list = []

n = int(input('Please input the number of course: '))
print('Please enter the credit and grade for each course: (Credit first, followed by the grade)')
for num in range(n):
    credit_input = int(input('Credit: '))
    grade_input = input('Grade: ')
    credit_list.append(credit_input)
    grade_list.append(grade_input)

Please input the number of course: 3
Please enter the credit and grade for each course: (Credit first, followed by the grade)
Credit: 3
Grade: A+
Credit: 3
Grade: A
Credit: 3
Grade: D


In [167]:
print('OUHK:')
stu1 = Student('Tang Ho Chun', 'M')
stu1.set_age_and_year(20)
stu1.set_major('IEDA')
stu1.gpa_calculate(credit_list, grade_list, 'ouhk')
print(f'name: {stu1.name}, gender: {stu1.gender}')
print(f'major: {stu1.major}, year: {stu1.year}')
print(f'sem gpa: {stu1.cga}')

OUHK:
Error!! A+ is not included in the grading system
name: Tang Ho Chun, gender: M
major: IEDA, year: 3
sem gpa: 0.0


## Lingnan U

In [168]:
credit_list = []
grade_list = []

n = int(input('Please input the number of course: '))
print('Please enter the credit and grade for each course: (Credit first, followed by the grade)')
for num in range(n):
    credit_input = int(input('Credit: '))
    grade_input = input('Grade: ')
    credit_list.append(credit_input)
    grade_list.append(grade_input)

Please input the number of course: 3
Please enter the credit and grade for each course: (Credit first, followed by the grade)
Credit: 3
Grade: A
Credit: 3
Grade: B
Credit: 3
Grade: D+


In [169]:
print('Lingnan U:')
stu1 = Student('Tang Ho Chun', 'M')
stu1.set_age_and_year(20)
stu1.set_major('IEDA')
stu1.gpa_calculate(credit_list, grade_list, 'lingu')
print(f'name: {stu1.name}, gender: {stu1.gender}')
print(f'major: {stu1.major}, year: {stu1.year}')
print(f'sem gpa: {stu1.cga}')

Lingnan U:
name: Tang Ho Chun, gender: M
major: IEDA, year: 3
sem gpa: 2.777


## HSUHK

In [170]:
print('HSUHK:')
stu1 = Student('Tang Ho Chun', 'M')
stu1.set_age_and_year(20)
stu1.set_major('IEDA')
stu1.gpa_calculate(credit_list, grade_list, 'hsuhk')
print(f'name: {stu1.name}, gender: {stu1.gender}')
print(f'major: {stu1.major}, year: {stu1.year}')
print(f'sem gpa: {stu1.cga}')

HSUHK:
name: Tang Ho Chun, gender: M
major: IEDA, year: 3
sem gpa: 2.767
